In [16]:
%load_ext autoreload
%autoreload 2

import pickle
import os
import numpy as np
import cv2
import time
import sys
import re
from copy import deepcopy
from collections import defaultdict
from itertools import islice
from tqdm import tqdm_notebook
from time import time
from easydict import EasyDict
from IPython.core.debugger import set_trace
from matplotlib import pyplot as plt
from warnings import filterwarnings

import torch
from torch import nn
import torch.nn.functional as F
from torch import autograd
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader

from mvn.datasets.human36m import Human36MMultiViewDataset
from mvn.utils.img import image_batch_to_numpy, denormalize_image, to_numpy 
from mvn.utils.multiview import project_3d_points_to_image_plane_without_distortion
from mvn.utils.vis import draw_2d_pose, display_pose, CONNECTIVITY_DICT
from mvn.utils.op import get_coord_volumes, integrate_tensor_3d_with_coordinates, unproject_heatmaps
from mvn.utils import cfg

from mvn.datasets import utils as dataset_utils

from train_decomposition import setup_human36m_dataloaders


%matplotlib inline

retval = {
    'subject_names': ['S1', 'S5', 'S6', 'S7', 'S8', 'S9', 'S11'],
    'camera_names': ['54138969', '55011271', '58860488', '60457274'],
    'action_names': [
        'Directions-1', 'Directions-2',
        'Discussion-1', 'Discussion-2',
        'Eating-1', 'Eating-2',
        'Greeting-1', 'Greeting-2',
        'Phoning-1', 'Phoning-2',
        'Posing-1', 'Posing-2',
        'Purchases-1', 'Purchases-2',
        'Sitting-1', 'Sitting-2',
        'SittingDown-1', 'SittingDown-2',
        'Smoking-1', 'Smoking-2',
        'TakingPhoto-1', 'TakingPhoto-2',
        'Waiting-1', 'Waiting-2',
        'Walking-1', 'Walking-2',
        'WalkingDog-1', 'WalkingDog-2',
        'WalkingTogether-1', 'WalkingTogether-2']
}

device = 'cuda:0' #torch.cuda.current_device()

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

def to_numpy(cuda_tensor):
    return cuda_tensor.detach().cpu().numpy()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
c = nn.Conv3d(64,64,kernel_size=[1,2,2],stride=[1,2,2])

In [31]:
t = torch.optim.Adam(c.parameters())

In [33]:
len(t.param_groups)

1

In [26]:
2**6

64

In [24]:
x = torch.randn(1,64,32,32,32)
c(x).shape

torch.Size([1, 64, 32, 16, 16])

In [14]:
isinstance(x, torch.Tensor)

True

In [11]:
x.sum((3,4)).shape

torch.Size([1, 64, 32])

In [2]:
# !pip install torchgeometry

# Calculate

In [3]:
config_path = './experiments/human36m/train/human36m_vol_image2lixel.yaml'
config = cfg.load_config(config_path)
num_joints = config.model.backbone.num_joints

In [4]:
from mvn.models.image2lixel import I2LModel
model = I2LModel(config, device=device).to(device)

Loading pretrained weights from: ./data/pose_resnet_4.5_pixels_human36m.pth
Successfully loaded pretrained weights for backbone


# Forward

In [5]:
train_loader, val_loader, _ = setup_human36m_dataloaders(config, is_train=True, distributed_train=False)

for batch in train_loader:
    break

(images_batch, 
keypoints_3d_gt, 
keypoints_3d_validity_gt, 
proj_matricies_batch) = dataset_utils.prepare_batch(batch, device)

heatmaps_pred, keypoints_2d_pred, cuboids_pred, base_points_pred = None, None, None, None
torch.cuda.empty_cache()

In [6]:
out = model(images_batch)

> /nfs/hpc2_storage/ibulygin/learnable-triangulation-pytorch/mvn/models/image2lixel.py(45)forward()
     43         _, _, _, _, pose_img_feat = self.backbone(images_batch.view(-1, 3, *image_shape))
     44         set_trace()
---> 45         joint_coord_img = self.pose_net(pose_img_feat)
     46 
     47         return joint_coord_img

ipdb> pose_img_feat.shape
torch.Size([1, 2048, 8, 8])
ipdb> c


# Compare backbones with volumetric baseline